In [1]:
import numpy as np
import scipy
import time
import math
from scipy.stats import norm

import mystic
from mystic.solvers import DifferentialEvolutionSolver, diffev2
from mystic.strategy import Best1Bin
from mystic.monitors import Monitor,VerboseMonitor
from copy import deepcopy

from tomography import *

from NestedForLoop import get_iterator
from pathlib import Path
from scipy.linalg import sqrtm

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import os
import glob

import pandas as pd

from scipy.optimize import least_squares

import fnmatch
from efficiencies import finding_file, get_channels_eff, set_raw_counts
from optimization import Optimizer, function_fidelity, FidelityResults
from constants import *

from densitymatrix import DensityMatrix, apply_unitary_to_dm, GHZ

from pathlib import Path
import fnmatch

In [2]:
######################################################################################################
#----- COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER -------
######################################################################################################

n_files=0
working_dir=r"C:\Users\nicol\Desktop\Experiment"
os.chdir(working_dir)

filenames = [i for i in glob.glob("StateTomography_2Layers_*")]
filenames.sort(key=os.path.getmtime)

index_to_file = {}

for index, filename in enumerate(filenames):
    os.chdir(f"{working_dir}\\{filename}")
    filenames_aux=[i for i in glob.glob("StateTomo_2Layers*")]
    for index_second, filenames_aux_second in enumerate(filenames_aux):
        index_to_file[n_files] = f"{filename}\\{filenames_aux_second}"
        n_files+=1
os.chdir(working_dir)

In [3]:
############################
#----- INITIALISATION------#
############################


players_ini=["Bran", "Arya", "Cersei", "Dany"]

column_start_ini = [8,12]

column_stop_ini = [12,16]

column_start_2_emissions_ini=[16,20]

column_stop_2_emissions_ini=[20,24]

qubit_number=2

error_runs=2

States = {}
    
Beam = ['low','high']

 
    
for i in range (2):
    
    os.chdir(working_dir)

    ## Defining the columns of the data file we want to use as data to reconstruct the density matrix (eg.: HH HV VH and VV basis)
    column_start =column_start_ini[i]
    column_stop = column_stop_ini[i]
    
    column_start_2_emissions= column_start_2_emissions_ini[i]
    column_stop_2_emissions= column_stop_2_emissions_ini[i]

    state_after=[]
    state_after_file=[]

    state_before=[]
    state_before_file=[]

    state = []
    state_file=[]

    xp_counts_corrected_with_eff=[]

    statetomo = []
    Fock_state = []
  
    U=[]
    bell_aux=[]
    target_ini=[]
    players = players_ini[0+i*2:2+i*2]
    

    #####################################################################
    #---------------------- STATE TOMOGRAPHY FOR THE FIRST PAIR----------
    #####################################################################
    for index in range(len(index_to_file)):
        os.chdir(f"{working_dir}\\{index_to_file[index]}\\")
        datafiles=[i for i in glob.glob("*")]

        ### Calculating the efficiencies of each detector
        
        print("Getting the efficiences")
        
        efficiencies=get_channels_eff(datafiles, column_start, column_stop, os.getcwd())

        ### Opening the data files and writing the data in counts_aux array
        
        print("Getting the counts")
        
        counts_aux=set_raw_counts(datafiles, qubit_number, column_start, column_stop, os.getcwd())
        xp_counts=np.array(np.transpose(counts_aux))
    
        counts_aux_2_emissions=set_raw_counts(datafiles, qubit_number, column_start_2_emissions, column_stop_2_emissions, os.getcwd())
        xp_counts_2_emissions=np.array(np.transpose(counts_aux_2_emissions))

        

        print("Doing the Tomography")
        
        
        statetomo.append(LRETomography(int(qubit_number),xp_counts, xp_counts_2_emissions))
        statetomo[-1].run(correct_eff=efficiencies)
        #     statetomo[-1].run(print_nc=False)
        xp_counts_corrected_with_eff.append(statetomo[-1].xp_counts)


        ## The 'e' and 'r' serve to distinguish between tomography before and after, respectively
        ## We want to save them in different arrays because we need them for different things
    #     elif index_to_file[index][-8]=='e':
    #         state_before.append(statetomo[-1])#.state)
    #         state_before_file.append(index_to_file[index])
    #         #print('\n Fast maximum likelihood estimation: \n', state_before[-1], '\n')

    #     elif index_to_file[index][-8]=='r':
    #         state_after.append(statetomo[-1])#.state)
    #         state_after_file.append(index_to_file[index])
    #         #print('\n Fast maximum likelihood estimation: \n', state_after[-1], '\n')

        state.append(statetomo[-1])#.state)
        state_file.append(index_to_file[index])
        
    states=state
    bell=(np.array([0,0,0,1])+np.array([1,0,0,0]))/np.sqrt(2)
    bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))
    fid=np.zeros((n_files))
    
    
    print("Optimizing the state")
    
    
    optimized_matrix=np.zeros((n_files,2**qubit_number,2**qubit_number), dtype='complex')
    
    guess=np.array([0, 0, 0])
    bounds=[(-np.pi,np.pi)]*3
    results = []

    opt=Optimizer(guess, function_fidelity, results=FidelityResults)

    for index in range(len(states)):
        result=opt.optimize(states[index].state, bell, bounds=bounds)
        results.append(result)
        
    for index in range(len(states)):
        target=bellmatrix
        target_ini.append(np.transpose(np.conjugate(bellmatrix)))
        states_file=state_file
        States['state_' + Beam[i] ] = results[-1].Density(results[-1].optimized_state.state)
        states= States['state_' + Beam[i]]
        
        print('file, fidelity, fidelity_mean, fidelity_std: ',
          states_file[index], np.round(states.fidelity(target_ini[-1]),5))
    
locals().update(States)

Getting the efficiences
Getting the counts
Doing the Tomography
Optimizing the state
file, fidelity, fidelity_mean, fidelity_std:  StateTomography_2Layers_20230330\StateTomo_2Layers (0.97468+0j)
Getting the efficiences
Getting the counts
Doing the Tomography
Optimizing the state
file, fidelity, fidelity_mean, fidelity_std:  StateTomography_2Layers_20230330\StateTomo_2Layers (0.95639-0j)


In [4]:
###########################################################
#------------ CREATION OF THE BEAMSPLITTER ---------------#
###########################################################

X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
PBS = np.bmat([[np.diag([1,1]),np.zeros((2,2))],[np.zeros((2,2)),X@Z]])
swap = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
PBS_swap = swap@PBS
Beamsplitter = np.kron(np.diag([1,1]),np.kron(PBS_swap,np.diag([1,1])))

In [5]:
##################################################
#------------ CREATION OF THE GHZ ---------------#
##################################################

print('Merging the two states')

GHZ_state = []
state_high.state_swap=[]
target_GHZ_without_PBS = []
swap = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]])
state_high_swap = swap@state_high.state
GHZ_state.append(GHZ(state_low.state,state_high_swap))
GHZ_state[-1].GHZ_before_BS()

print('Applying the Beam splitter')

GHZ_state[-1].BeamSplitter()

Merging the two states
Applying the Beam splitter


In [6]:
##########################################################
#------------ CREATION OF THE TARGET STATE---------------#
##########################################################

target_bell = np.tensordot(((np.array([0,0,0,1])+np.array([1,0,0,0]))/np.sqrt(2)),((np.array([0,0,0,1])+np.array([1,0,0,0]))/np.sqrt(2)),axes=0)
swap_target_bell = swap@target_bell
target_GHZ_without_PBS = np.kron(target_bell,swap_target_bell)
target_GHZ=Beamsplitter@target_GHZ_without_PBS

print('file, fidelity',states_file, np.round(GHZ_state[-1].fidelity(target_GHZ),5))


file, fidelity ['StateTomography_2Layers_20230330\\StateTomo_2Layers'] (0.90881+0.02495j)
